In [2]:
from scipy.fft import fft
import scipy
import os
import numpy as np
from numpy import set_printoptions
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from scipy.stats import mannwhitneyu
import h5py
import math
import pandas as pd
import random
import json
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')
!pip install mat73
DATASET_DIR = '/content/drive/MyDrive/IMUyAccelerometer/Data'
import mat73
from scipy.signal import butter,filtfilt

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
#CODE UTILITIES

#return column of non-pandas matrix
def column(matrix, i):
    return [row[i] for row in matrix]

#Set up Nyquist Frequency and Butterworth lowpass filter to remove noise
fs = 100.0 
cutoff = 5     
nyq = 0.5 * fs  
order = 3

def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data, padlen=6)
    return y

#define a few different feature selection methods to test
def featureSelection(features, k, labels, method):
    X = features.values
    Y = labels
    if method == 0:
      #RFE
      model = LogisticRegression(solver='lbfgs')
      rfe = RFE(model, n_features_to_select=k)
      fit = rfe.fit(X, Y)
    
    if method == 1:
      #PCA
      pca = PCA(n_components=k)
      fit = pca.fit(X)

    if method == 2:
      #Univariate
      test = SelectKBest(score_func=f_classif, k=k)
      fit = test.fit(X, Y)
      set_printoptions(precision=3)
      print(fit.scores_)
      features = fit.transform(X)

In [45]:
#load all LLA data
s01 = mat73.loadmat(DATASET_DIR+'/S01.mat')
s02 = mat73.loadmat(DATASET_DIR+'/S02.mat')
s03 = mat73.loadmat(DATASET_DIR+'/S03.mat')
s04 = mat73.loadmat(DATASET_DIR+'/S04.mat')
s05 = mat73.loadmat(DATASET_DIR+'/S05.mat')
s06 = mat73.loadmat(DATASET_DIR+'/S06.mat')
s07 = mat73.loadmat(DATASET_DIR+'/S07.mat')
s08 = mat73.loadmat(DATASET_DIR+'/S08.mat')
s09 = mat73.loadmat(DATASET_DIR+'/S09.mat')
s10 = mat73.loadmat(DATASET_DIR+'/S10.mat')
s11 = mat73.loadmat(DATASET_DIR+'/S11.mat')
s12 = mat73.loadmat(DATASET_DIR+'/S12.mat')
s13 = mat73.loadmat(DATASET_DIR+'/S13.mat')
s14 = mat73.loadmat(DATASET_DIR+'/S14.mat')
s15 = mat73.loadmat(DATASET_DIR+'/S15.mat')
s16 = mat73.loadmat(DATASET_DIR+'/S16.mat')
s17 = mat73.loadmat(DATASET_DIR+'/S17.mat')

In [51]:
#Load all control data
c01 = mat73.loadmat(DATASET_DIR+'/C01.mat')
c02 = mat73.loadmat(DATASET_DIR+'/C02.mat')
c03 = mat73.loadmat(DATASET_DIR+'/C03.mat')
c04 = mat73.loadmat(DATASET_DIR+'/C04.mat')
c05 = mat73.loadmat(DATASET_DIR+'/C05.mat')
c06 = mat73.loadmat(DATASET_DIR+'/C06.mat')
c07 = mat73.loadmat(DATASET_DIR+'/C07.mat')
c08 = mat73.loadmat(DATASET_DIR+'/C08.mat')
c09 = mat73.loadmat(DATASET_DIR+'/C09.mat')
c10 = mat73.loadmat(DATASET_DIR+'/C10.mat')
c11 = mat73.loadmat(DATASET_DIR+'/C11.mat')
c12 = mat73.loadmat(DATASET_DIR+'/C12.mat')
c13 = mat73.loadmat(DATASET_DIR+'/C13.mat')
c14 = mat73.loadmat(DATASET_DIR+'/C14.mat')

In [46]:
#Isolate the sections of the data we need - See the Instructions.pdf from the original dataset organization
#The variable K exists in case of low-RAM runs. The full dataset requires ~64 GB of RAM and >4 hours to perform calculations
# to enable the full run, replace k with len(x[i])
x = [s01['SubjectData']['IMU']['Bouts']['Data'], s02['SubjectData']['IMU']['Bouts']['Data'], s03['SubjectData']['IMU']['Bouts']['Data'], s04['SubjectData']['IMU']['Bouts']['Data'], s05['SubjectData']['IMU']['Bouts']['Data'], s06['SubjectData']['IMU']['Bouts']['Data'], s07['SubjectData']['IMU']['Bouts']['Data'], s08['SubjectData']['IMU']['Bouts']['Data'], s09['SubjectData']['IMU']['Bouts']['Data'], s10['SubjectData']['IMU']['Bouts']['Data'], s11['SubjectData']['IMU']['Bouts']['Data'], s12['SubjectData']['IMU']['Bouts']['Data'], s13['SubjectData']['IMU']['Bouts']['Data'], s14['SubjectData']['IMU']['Bouts']['Data'], s15['SubjectData']['IMU']['Bouts']['Data'], s16['SubjectData']['IMU']['Bouts']['Data'], s17['SubjectData']['IMU']['Bouts']['Data'] ]

y = [s01['SubjectData']['IMU']['Bouts']['Timestamps'], s02['SubjectData']['IMU']['Bouts']['Timestamps'], s03['SubjectData']['IMU']['Bouts']['Timestamps'], s04['SubjectData']['IMU']['Bouts']['Timestamps'], s05['SubjectData']['IMU']['Bouts']['Timestamps'], s06['SubjectData']['IMU']['Bouts']['Timestamps'], s07['SubjectData']['IMU']['Bouts']['Timestamps'], s08['SubjectData']['IMU']['Bouts']['Timestamps'], s09['SubjectData']['IMU']['Bouts']['Timestamps'], s10['SubjectData']['IMU']['Bouts']['Timestamps'], s11['SubjectData']['IMU']['Bouts']['Timestamps'], s12['SubjectData']['IMU']['Bouts']['Timestamps'], s13['SubjectData']['IMU']['Bouts']['Timestamps'], s14['SubjectData']['IMU']['Bouts']['Timestamps'], s15['SubjectData']['IMU']['Bouts']['Timestamps'], s16['SubjectData']['IMU']['Bouts']['Timestamps'], s17['SubjectData']['IMU']['Bouts']['Timestamps'] ]
print(s01['SubjectData']['IMU']['Bouts']['DataHeaders'])
headers = s01['SubjectData']['IMU']['Bouts']['DataHeaders']
data = []
time = []
k=20
for i in range(len(x)): 
  for j in range(k):
    data.append((x[i][j][0]))
    time.append(y[i][j][0])
  

print(len(data))

['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z', 'Temperature', 'Gyroscope X', 'Gyroscope Y', 'Gyroscope Z', 'Magnetometer X', 'Magnetometer Y', 'Magnetometer Z']
340


In [55]:
x = [c01['SubjectData']['IMU']['Bouts']['Data'], c02['SubjectData']['IMU']['Bouts']['Data'], c03['SubjectData']['IMU']['Bouts']['Data'], c04['SubjectData']['IMU']['Bouts']['Data'], c05['SubjectData']['IMU']['Bouts']['Data'], c06['SubjectData']['IMU']['Bouts']['Data'], c07['SubjectData']['IMU']['Bouts']['Data'], c08['SubjectData']['IMU']['Bouts']['Data'], c09['SubjectData']['IMU']['Bouts']['Data'], c10['SubjectData']['IMU']['Bouts']['Data'], c11['SubjectData']['IMU']['Bouts']['Data'], c12['SubjectData']['IMU']['Bouts']['Data'], c13['SubjectData']['IMU']['Bouts']['Data'], c14['SubjectData']['IMU']['Bouts']['Data'] ]
y = [c01['SubjectData']['IMU']['Bouts']['Timestamps'], c02['SubjectData']['IMU']['Bouts']['Timestamps'], c03['SubjectData']['IMU']['Bouts']['Timestamps'], c04['SubjectData']['IMU']['Bouts']['Timestamps'], c05['SubjectData']['IMU']['Bouts']['Timestamps'], c06['SubjectData']['IMU']['Bouts']['Timestamps'], c07['SubjectData']['IMU']['Bouts']['Timestamps'], c08['SubjectData']['IMU']['Bouts']['Timestamps'], c09['SubjectData']['IMU']['Bouts']['Timestamps'], c10['SubjectData']['IMU']['Bouts']['Timestamps'], c11['SubjectData']['IMU']['Bouts']['Timestamps'], c12['SubjectData']['IMU']['Bouts']['Timestamps'], c13['SubjectData']['IMU']['Bouts']['Timestamps'], c14['SubjectData']['IMU']['Bouts']['Timestamps'] ]
print(c01['SubjectData']['IMU']['Bouts']['DataHeaders'])
headers = c01['SubjectData']['IMU']['Bouts']['DataHeaders']
data = []
time = []
k=24
for i in range(len(x)): 
  for j in range(k):
    data.append((x[i][j][0]))
    time.append(y[i][j][0])
print(len(data))

['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z', 'Temperature', 'Gyroscope X', 'Gyroscope Y', 'Gyroscope Z', 'Magnetometer X', 'Magnetometer Y', 'Magnetometer Z']
336


In [56]:
#clean up our data, remove incomplete data sets and drop unnessecary columns
# then, apply the butterworth filter to Accelerometer data
#We also send our cleaned up data to a pandas DataFrame for easier manipulation

datax = []
j = -1
for i in range(len(data)):
  #cull all incomplete data / non waveform data
  if type(data[i][0]) is np.float64 or len(data[i][0])<10 or \
   sum(data[i][0])==0:
    continue
  datax.append(pd.DataFrame(data[i]))
  datax[j].columns = headers
  datax[j]['Time'] = time[i]
print(len(datax))
# butterworth filter on all Accelerometer data, drop unneccesary temp data
for k in range(len(datax)):
  x = butter_lowpass_filter(datax[k].iloc[:,0], cutoff, fs, order)
  datax[k].iloc[:,0] = x
  y = butter_lowpass_filter(datax[k].iloc[:,1], cutoff, fs, order)
  datax[k].iloc[:,0] = y
  z = butter_lowpass_filter(datax[k].iloc[:,2], cutoff, fs, order)
  datax[k].iloc[:,0] = z
  datax[k].drop(datax[k].columns[[3]], axis=1, inplace=True)

datac = []
j = -1
for i in range(len(data)):
  #cull all incomplete data / non waveform data
  if type(data[i][0]) is np.float64 or len(data[i][0])<10 or \
   sum(data[i][0])==0:
    continue
  datac.append(pd.DataFrame(data[i]))
  datac[j].columns = headers
  datac[j]['Time'] = time[i]
print(len(datac))
# butterworth filter on all Accelerometer data, drop unneccesary temp data
for k in range(len(datac)):
  x = butter_lowpass_filter(datac[k].iloc[:,0], cutoff, fs, order)
  datac[k].iloc[:,0] = x
  y = butter_lowpass_filter(datac[k].iloc[:,1], cutoff, fs, order)
  datac[k].iloc[:,0] = y
  z = butter_lowpass_filter(datac[k].iloc[:,2], cutoff, fs, order)
  datac[k].iloc[:,0] = z
  datac[k].drop(datac[k].columns[[3]], axis=1, inplace=True)


333
333


In [50]:
#calculate jerk from acceleration data
# Jerk - the third derivative of position, or first derivative of Acceleration, describes 
# change in Acceleration over time. This part of the process takes the longest of all calculations.
for k in range(len(datax)):
  jerkx = []
  jerky = []
  jerkz = []
  jerkx.append(0)
  jerky.append(0)
  jerkz.append(0)
  for i in range(len(datax[k].iloc[:,1])-1):
    jerkx.append((datax[k].iloc[:,0][i]-datax[k].iloc[:,0][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
    jerky.append((datax[k].iloc[:,1][i]-datax[k].iloc[:,1][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
    jerkz.append((datax[k].iloc[:,2][i]-datax[k].iloc[:,2][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
  datax[k]['Jerk X'] = jerkx
  datax[k]['Jerk Y'] = jerky
  datax[k]['Jerk Z'] = jerkz 

<ipython-input-50-194321ca3392>:10: RuntimeWarning: divide by zero encountered in double_scalars
  jerkx.append((datax[k].iloc[:,0][i]-datax[k].iloc[:,0][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
<ipython-input-50-194321ca3392>:11: RuntimeWarning: divide by zero encountered in double_scalars
  jerky.append((datax[k].iloc[:,1][i]-datax[k].iloc[:,1][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
<ipython-input-50-194321ca3392>:12: RuntimeWarning: divide by zero encountered in double_scalars
  jerkz.append((datax[k].iloc[:,2][i]-datax[k].iloc[:,2][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
<ipython-input-50-194321ca3392>:12: RuntimeWarning: invalid value encountered in double_scalars
  jerkz.append((datax[k].iloc[:,2][i]-datax[k].iloc[:,2][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
<ipython-input-50-194321ca3392>:11: RuntimeWarning: invalid value encountered in double_scalars
  jerky.append((datax[k].iloc[:,1][i]

In [57]:
#calculate jerk on control dataset
for k in range(len(datac)):
  jerkx = []
  jerky = []
  jerkz = []
  jerkx.append(0)
  jerky.append(0)
  jerkz.append(0)
  for i in range(len(datac[k].iloc[:,1])-1):
    jerkx.append((datac[k].iloc[:,0][i]-datac[k].iloc[:,0][i+1])/(datac[k].loc[:,'Time'][i]-datac[k].loc[:,'Time'][i+1]))
    jerky.append((datac[k].iloc[:,1][i]-datac[k].iloc[:,1][i+1])/(datac[k].loc[:,'Time'][i]-datac[k].loc[:,'Time'][i+1]))
    jerkz.append((datac[k].iloc[:,2][i]-datac[k].iloc[:,2][i+1])/(datac[k].loc[:,'Time'][i]-datac[k].loc[:,'Time'][i+1]))
  datac[k]['Jerk X'] = jerkx
  datac[k]['Jerk Y'] = jerky
  datac[k]['Jerk Z'] = jerkz 

In [ ]:
# Calculate Accelerometer and Jerk Magnitudes on Full Dataset
# Magnitude of a vector = sqrt(x^2+y^2+z^2)
#This is likely the 2nd longest operation in the whole thing
for k in range(len(datac)):
  accelmag = []
  jerkmag = []
  for i in range(len(datac[k].iloc[:,1])):
    accelmag.append( math.sqrt(datac[k].iloc[:,0][i]**2 + datac[k].iloc[:,1][i]**2 + datac[k].iloc[:,2][i]**2) )
    jerkmag.append( math.sqrt(datac[k].iloc[:,10][i]**2 + datac[k].iloc[:,11][i]**2 + datac[k].iloc[:,12][i]**2) )
  datac[k]['Accel Mag'] = accelmag
  datac[k]['Jerk Mag'] = jerkmag

for k in range(len(datax)):
  accelmag = []
  jerkmag = []
  for i in range(len(datax[k].iloc[:,1])):
    accelmag.append( math.sqrt(datax[k].iloc[:,0][i]**2 + datax[k].iloc[:,1][i]**2 + datax[k].iloc[:,2][i]**2) )
    jerkmag.append( math.sqrt(datax[k].iloc[:,10][i]**2 + datax[k].iloc[:,11][i]**2 + datax[k].iloc[:,12][i]**2) )
  datax[k]['Accel Mag'] = accelmag
  datax[k]['Jerk Mag'] = jerkmag


In [ ]:
# OPTIONALLY, SAVE DATA TO CSVs TO RE-IMPORT FOR LATER EVALUATION
for k in range(len(datac)):
  filename = Path(DATASET_DIR+'/control/'+str(k)+'.csv')
  datac[k].to_csv(filename)

for k in range(len(datax)):
  filename = Path(DATASET_DIR+'/lla/'+str(k)+'.csv')
  datax[k].to_csv(filename)

In [3]:
# OPTIONALLY, IMPORT DATA BACK FROM CSVs TO SAVE TIME/RAM on future runs
k = 0
datax = []
filenameL = Path(DATASET_DIR+'/lla/'+str(k)+'.csv')
for k in range(169):
  df = pd.read_csv(filenameL)
  datax.append(df)

k = 0
datac = []
filenameC = Path(DATASET_DIR+'/control/'+str(k)+'.csv')
for k in range(236):
  df = pd.read_csv(filenameC)
  datac.append(df)


In [ ]:
#Calculate Sliding Windows. Divy up the data into sliding windows of size 100 (1s of data) + a 33 frame offset
print(len(datax))
temp = datax
datax = []
for k in range(len(temp)):
  c = 1
  while len(temp[k]) - c >=100:
    datax.append(temp.iloc[c,c+100])
    c+=33
print(len(datax))
print(len(datac))
temp = datac
datac = []
for k in range(len(temp)):
  c = 1
  while len(temp[k]) - c >=100:
    datac.append(temp.iloc[c,c+100])
    c+=33
print(len(datac))

In [13]:
# get wavelet/fft features. These are complex numbers so we will need to use the magnitude 
# instead of the raw fft values in order to perform evaluations on them
fft = []

#fft on LLA dataset
for k in range(len(datax)):
  df = pd.DataFrame()
  df['accXfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Accelerometer X']))]
  df['accYfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Accelerometer Y']))]
  df['accZfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Accelerometer Z']))]
  df['gyroXfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Gyroscope X']))]
  df['gyroYfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Gyroscope Y']))]
  df['gyroZfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Gyroscope Z']))]
  df['magXfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Magnetometer X']))]
  df['magYfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Magnetometer Y']))]
  df['magZfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Magnetometer Z']))]
  df['jerkXfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Jerk X']))]
  df['jerkYfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Jerk Y']))]
  df['jerkZfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Jerk Z']))]
  df['magAccelfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Accel Mag']))]
  df['magJerkfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Jerk Mag']))]
  fft.append(df)
for k in range(len(datac)):
  df = pd.DataFrame()
  df['accXfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Accelerometer X']))]
  df['accYfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Accelerometer Y']))]
  df['accZfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Accelerometer Z']))]
  df['gyroXfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Gyroscope X']))]
  df['gyroYfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Gyroscope Y']))]
  df['gyroZfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Gyroscope Z']))]
  df['magXfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Magnetometer X']))]
  df['magYfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Magnetometer Y']))]
  df['magZfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Magnetometer Z']))]
  df['jerkXfft'] = [abs(ele) for ele in fft(np.array(datax[k].loc[:,'Jerk X']))]
  df['jerkYfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Jerk Y']))]
  df['jerkZfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Jerk Z']))]
  df['magAccelfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Accel Mag']))]
  df['magJerkfft'] = [abs(ele) for ele in fft(np.array(datac[k].loc[:,'Jerk Mag']))]
  fft.append(df)



KeyError: ignored

In [14]:
#Get features from the time domain. This is also where we apply our labels to each waveform 
features = []
for k in range(len(datax)):
  item = [0, min(datax[k].loc[:,'Accelerometer X']), max(datax[k].loc[:,'Accelerometer X']), np.mean(datax[k].loc[:,'Accelerometer X']), np.std(datax[k].loc[:,'Accelerometer X'], axis=0),
          min(datax[k].loc[:,'Accelerometer Y']), max(datax[k].loc[:,'Accelerometer Y']), np.mean(datax[k].loc[:,'Accelerometer Y']), np.std(datax[k].loc[:,'Accelerometer Y'], axis=0),
          min(datax[k].loc[:,'Accelerometer Z']), max(datax[k].loc[:,'Accelerometer Z']), np.mean(datax[k].loc[:,'Accelerometer Z']), np.std(datax[k].loc[:,'Accelerometer Z'], axis=0),
          min(datax[k].loc[:,'Gyroscope X']), max(datax[k].loc[:,'Gyroscope X']), np.mean(datax[k].loc[:,'Gyroscope X']), np.std(datax[k].loc[:,'Gyroscope X'], axis=0),
          min(datax[k].loc[:,'Gyroscope Y']), max(datax[k].loc[:,'Gyroscope Y']), np.mean(datax[k].loc[:,'Gyroscope Y']), np.std(datax[k].loc[:,'Gyroscope Y'], axis=0),
          min(datax[k].loc[:,'Gyroscope Z']), max(datax[k].loc[:,'Gyroscope Z']), np.mean(datax[k].loc[:,'Gyroscope Z']), np.std(datax[k].loc[:,'Gyroscope Z'], axis=0),
          min(datax[k].loc[:,'Magnetometer X']), max(datax[k].loc[:,'Magnetometer X']), np.mean(datax[k].loc[:,'Magnetometer X']), np.std(datax[k].loc[:,'Magnetometer X'], axis=0),
          min(datax[k].loc[:,'Magnetometer Y']), max(datax[k].loc[:,'Magnetometer Y']), np.mean(datax[k].loc[:,'Magnetometer Y']), np.std(datax[k].loc[:,'Magnetometer Y'], axis=0),
          min(datax[k].loc[:,'Magnetometer Z']), max(datax[k].loc[:,'Magnetometer Z']), np.mean(datax[k].loc[:,'Magnetometer Z']), np.std(datax[k].loc[:,'Magnetometer Z'], axis=0),
          min(datax[k].loc[:,'Jerk X']), max(datax[k].loc[:,'Jerk X']), np.mean(datax[k].loc[:,'Jerk X']), np.std(datax[k].loc[:,'Jerk X'], axis=0),
          min(datax[k].loc[:,'Jerk Y']), max(datax[k].loc[:,'Jerk Y']), np.mean(datax[k].loc[:,'Jerk Y']), np.std(datax[k].loc[:,'Jerk Y'], axis=0),
          min(datax[k].loc[:,'Jerk Z']), max(datax[k].loc[:,'Jerk Z']), np.mean(datax[k].loc[:,'Jerk Z']), np.std(datax[k].loc[:,'Jerk Z'], axis=0),
          min(datax[k].loc[:,'Accel Mag']), max(datax[k].loc[:,'Accel Mag']), np.mean(datax[k].loc[:,'Accel Mag']), np.std(datax[k].loc[:,'Accel Mag'], axis=0),
          min(datax[k].loc[:,'Jerk Mag']), max(datax[k].loc[:,'Jerk Mag']), np.mean(datax[k].loc[:,'Jerk Mag']), np.std(datax[k].loc[:,'Jerk Mag'], axis=0),]
  features.append(item)
for k in range(len(datac)):
  item = [1, min(datac[k].loc[:,'Accelerometer X']), max(datac[k].loc[:,'Accelerometer X']), np.mean(datac[k].loc[:,'Accelerometer X']), np.std(datac[k].loc[:,'Accelerometer X'], axis=0),
          min(datac[k].loc[:,'Accelerometer Y']), max(datac[k].loc[:,'Accelerometer Y']), np.mean(datac[k].loc[:,'Accelerometer Y']), np.std(datac[k].loc[:,'Accelerometer Y'], axis=0),
          min(datac[k].loc[:,'Accelerometer Z']), max(datac[k].loc[:,'Accelerometer Z']), np.mean(datac[k].loc[:,'Accelerometer Z']), np.std(datac[k].loc[:,'Accelerometer Z'], axis=0),
          min(datac[k].loc[:,'Gyroscope X']), max(datac[k].loc[:,'Gyroscope X']), np.mean(datac[k].loc[:,'Gyroscope X']), np.std(datac[k].loc[:,'Gyroscope X'], axis=0),
          min(datac[k].loc[:,'Gyroscope Y']), max(datac[k].loc[:,'Gyroscope Y']), np.mean(datac[k].loc[:,'Gyroscope Y']), np.std(datac[k].loc[:,'Gyroscope Y'], axis=0),
          min(datac[k].loc[:,'Gyroscope Z']), max(datac[k].loc[:,'Gyroscope Z']), np.mean(datac[k].loc[:,'Gyroscope Z']), np.std(datac[k].loc[:,'Gyroscope Z'], axis=0),
          min(datac[k].loc[:,'Magnetometer X']), max(datac[k].loc[:,'Magnetometer X']), np.mean(datac[k].loc[:,'Magnetometer X']), np.std(datac[k].loc[:,'Magnetometer X'], axis=0),
          min(datac[k].loc[:,'Magnetometer Y']), max(datac[k].loc[:,'Magnetometer Y']), np.mean(datac[k].loc[:,'Magnetometer Y']), np.std(datac[k].loc[:,'Magnetometer Y'], axis=0),
          min(datac[k].loc[:,'Magnetometer Z']), max(datac[k].loc[:,'Magnetometer Z']), np.mean(datac[k].loc[:,'Magnetometer Z']), np.std(datac[k].loc[:,'Magnetometer Z'], axis=0),
          min(datac[k].loc[:,'Jerk X']), max(datac[k].loc[:,'Jerk X']), np.mean(datac[k].loc[:,'Jerk X']), np.std(datac[k].loc[:,'Jerk X'], axis=0),
          min(datac[k].loc[:,'Jerk Y']), max(datac[k].loc[:,'Jerk Y']), np.mean(datac[k].loc[:,'Jerk Y']), np.std(datac[k].loc[:,'Jerk Y'], axis=0),
          min(datac[k].loc[:,'Jerk Z']), max(datac[k].loc[:,'Jerk Z']), np.mean(datac[k].loc[:,'Jerk Z']), np.std(datac[k].loc[:,'Jerk Z'], axis=0),
          min(datac[k].loc[:,'Accel Mag']), max(datac[k].loc[:,'Accel Mag']), np.mean(datac[k].loc[:,'Accel Mag']), np.std(datac[k].loc[:,'Accel Mag'], axis=0),
          min(datac[k].loc[:,'Jerk Mag']), max(datac[k].loc[:,'Jerk Mag']), np.mean(datac[k].loc[:,'Jerk Mag']), np.std(datac[k].loc[:,'Jerk Mag'], axis=0),]
  features.append(item)


In [ ]:
#Calculate and append fft features 
features = []
for k in range(len(datax)):
  item = [0, min(datax[k].loc[:,'Accelerometer X']), max(datax[k].loc[:,'Accelerometer X']), np.mean(datax[k].loc[:,'Accelerometer X']), np.std(datax[k].loc[:,'Accelerometer X'], axis=0),
          min(datax[k].loc[:,'Accelerometer Y']), max(datax[k].loc[:,'Accelerometer Y']), np.mean(datax[k].loc[:,'Accelerometer Y']), np.std(datax[k].loc[:,'Accelerometer Y'], axis=0),
          min(datax[k].loc[:,'Accelerometer Z']), max(datax[k].loc[:,'Accelerometer Z']), np.mean(datax[k].loc[:,'Accelerometer Z']), np.std(datax[k].loc[:,'Accelerometer Z'], axis=0),
          min(datax[k].loc[:,'Gyroscope X']), max(datax[k].loc[:,'Gyroscope X']), np.mean(datax[k].loc[:,'Gyroscope X']), np.std(datax[k].loc[:,'Gyroscope X'], axis=0),
          min(datax[k].loc[:,'Gyroscope Y']), max(datax[k].loc[:,'Gyroscope Y']), np.mean(datax[k].loc[:,'Gyroscope Y']), np.std(datax[k].loc[:,'Gyroscope Y'], axis=0),
          min(datax[k].loc[:,'Gyroscope Z']), max(datax[k].loc[:,'Gyroscope Z']), np.mean(datax[k].loc[:,'Gyroscope Z']), np.std(datax[k].loc[:,'Gyroscope Z'], axis=0),
          min(datax[k].loc[:,'Magnetometer X']), max(datax[k].loc[:,'Magnetometer X']), np.mean(datax[k].loc[:,'Magnetometer X']), np.std(datax[k].loc[:,'Magnetometer X'], axis=0),
          min(datax[k].loc[:,'Magnetometer Y']), max(datax[k].loc[:,'Magnetometer Y']), np.mean(datax[k].loc[:,'Magnetometer Y']), np.std(datax[k].loc[:,'Magnetometer Y'], axis=0),
          min(datax[k].loc[:,'Magnetometer Z']), max(datax[k].loc[:,'Magnetometer Z']), np.mean(datax[k].loc[:,'Magnetometer Z']), np.std(datax[k].loc[:,'Magnetometer Z'], axis=0),
          min(datax[k].loc[:,'Jerk X']), max(datax[k].loc[:,'Jerk X']), np.mean(datax[k].loc[:,'Jerk X']), np.std(datax[k].loc[:,'Jerk X'], axis=0),
          min(datax[k].loc[:,'Jerk Y']), max(datax[k].loc[:,'Jerk Y']), np.mean(datax[k].loc[:,'Jerk Y']), np.std(datax[k].loc[:,'Jerk Y'], axis=0),
          min(datax[k].loc[:,'Jerk Z']), max(datax[k].loc[:,'Jerk Z']), np.mean(datax[k].loc[:,'Jerk Z']), np.std(datax[k].loc[:,'Jerk Z'], axis=0),
          min(datax[k].loc[:,'Accel Mag']), max(datax[k].loc[:,'Accel Mag']), np.mean(datax[k].loc[:,'Accel Mag']), np.std(datax[k].loc[:,'Accel Mag'], axis=0),
          min(datax[k].loc[:,'Jerk Mag']), max(datax[k].loc[:,'Jerk Mag']), np.mean(datax[k].loc[:,'Jerk Mag']), np.std(datax[k].loc[:,'Jerk Mag'], axis=0),]
  features.append(item)
for k in range(len(datac)):
  item = [1, min(datac[k].loc[:,'Accelerometer X']), max(datac[k].loc[:,'Accelerometer X']), np.mean(datac[k].loc[:,'Accelerometer X']), np.std(datac[k].loc[:,'Accelerometer X'], axis=0),
          min(datac[k].loc[:,'Accelerometer Y']), max(datac[k].loc[:,'Accelerometer Y']), np.mean(datac[k].loc[:,'Accelerometer Y']), np.std(datac[k].loc[:,'Accelerometer Y'], axis=0),
          min(datac[k].loc[:,'Accelerometer Z']), max(datac[k].loc[:,'Accelerometer Z']), np.mean(datac[k].loc[:,'Accelerometer Z']), np.std(datac[k].loc[:,'Accelerometer Z'], axis=0),
          min(datac[k].loc[:,'Gyroscope X']), max(datac[k].loc[:,'Gyroscope X']), np.mean(datac[k].loc[:,'Gyroscope X']), np.std(datac[k].loc[:,'Gyroscope X'], axis=0),
          min(datac[k].loc[:,'Gyroscope Y']), max(datac[k].loc[:,'Gyroscope Y']), np.mean(datac[k].loc[:,'Gyroscope Y']), np.std(datac[k].loc[:,'Gyroscope Y'], axis=0),
          min(datac[k].loc[:,'Gyroscope Z']), max(datac[k].loc[:,'Gyroscope Z']), np.mean(datac[k].loc[:,'Gyroscope Z']), np.std(datac[k].loc[:,'Gyroscope Z'], axis=0),
          min(datac[k].loc[:,'Magnetometer X']), max(datac[k].loc[:,'Magnetometer X']), np.mean(datac[k].loc[:,'Magnetometer X']), np.std(datac[k].loc[:,'Magnetometer X'], axis=0),
          min(datac[k].loc[:,'Magnetometer Y']), max(datac[k].loc[:,'Magnetometer Y']), np.mean(datac[k].loc[:,'Magnetometer Y']), np.std(datac[k].loc[:,'Magnetometer Y'], axis=0),
          min(datac[k].loc[:,'Magnetometer Z']), max(datac[k].loc[:,'Magnetometer Z']), np.mean(datac[k].loc[:,'Magnetometer Z']), np.std(datac[k].loc[:,'Magnetometer Z'], axis=0),
          min(datac[k].loc[:,'Jerk X']), max(datac[k].loc[:,'Jerk X']), np.mean(datac[k].loc[:,'Jerk X']), np.std(datac[k].loc[:,'Jerk X'], axis=0),
          min(datac[k].loc[:,'Jerk Y']), max(datac[k].loc[:,'Jerk Y']), np.mean(datac[k].loc[:,'Jerk Y']), np.std(datac[k].loc[:,'Jerk Y'], axis=0),
          min(datac[k].loc[:,'Jerk Z']), max(datac[k].loc[:,'Jerk Z']), np.mean(datac[k].loc[:,'Jerk Z']), np.std(datac[k].loc[:,'Jerk Z'], axis=0),
          min(datac[k].loc[:,'Accel Mag']), max(datac[k].loc[:,'Accel Mag']), np.mean(datac[k].loc[:,'Accel Mag']), np.std(datac[k].loc[:,'Accel Mag'], axis=0),
          min(datac[k].loc[:,'Jerk Mag']), max(datac[k].loc[:,'Jerk Mag']), np.mean(datac[k].loc[:,'Jerk Mag']), np.std(datac[k].loc[:,'Jerk Mag'], axis=0),]
  features.append(item)


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
df = pd.DataFrame(features)
x = df.sample(frac=1)
y = x.iloc[:,0]
x = x.drop(labels=0, axis=1)

           1         2         3         4         5         6         7   \
139 -0.806563  1.634289  0.608125  0.177891 -4.783203  1.035010 -0.753776   
116 -0.806563  1.634289  0.608125  0.177891 -4.783203  1.035010 -0.753776   
106 -0.806563  1.634289  0.608125  0.177891 -4.783203  1.035010 -0.753776   
221 -1.597056  2.395973  0.815542  0.275326 -9.824219  2.314941 -0.558163   
366 -1.597056  2.395973  0.815542  0.275326 -9.824219  2.314941 -0.558163   
..        ...       ...       ...       ...       ...       ...       ...   
289 -1.597056  2.395973  0.815542  0.275326 -9.824219  2.314941 -0.558163   
206 -1.597056  2.395973  0.815542  0.275326 -9.824219  2.314941 -0.558163   
109 -0.806563  1.634289  0.608125  0.177891 -4.783203  1.035010 -0.753776   
325 -1.597056  2.395973  0.815542  0.275326 -9.824219  2.314941 -0.558163   
84  -0.806563  1.634289  0.608125  0.177891 -4.783203  1.035010 -0.753776   

           8         9         10  ...           47            48        49

In [44]:
#Train Random Forest Classifier

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
yval_pred=clf.predict(X_val)
from sklearn import metrics
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_val, yval_pred))

Accuracy: 0.5327868852459017
